In [1]:
import pandas as pd
import os
import re
import unicodedata
from rapidfuzz import fuzz, process as fuzzprocess
import numpy as np

In [2]:

_re_non_alnum = re.compile(r"[^0-9a-z]+")
_re_multi_unders = re.compile(r"_+")

def normalizar_texto(texto: str) -> str:
    """Normaliza texto : sin acentos, minúsculas, underscores limpios."""
    if not isinstance(texto, str):
        return texto

    texto = unicodedata.normalize("NFKD", texto)
    texto = "".join(c for c in texto if not unicodedata.combining(c))
    texto = texto.lower()

    texto = _re_non_alnum.sub("_", texto)
    texto = _re_multi_unders.sub("_", texto)
    return texto.strip("_")

In [3]:
""""

Vehículos homologados antes de ~2014 COLUMNA AM.
Vehículos homologados 2014 en adelante COLUMNA  AH.
Vehículos 100 % eléctricos o de hidrógeno = 0"""

"""
Si la columna “Año modelo” ≥ 2015 → usa columna AH (WLTP)
Si la columna “Año modelo” ≤ 2014 → usa columna AM (NEDC o FTP-75)
Si “Tecnología” = BEV (Battery Electric Vehicle) o FCEV → emisiones = 0

"""

'\nSi la columna “Año modelo” ≥ 2015 → usa columna AH (WLTP)\nSi la columna “Año modelo” ≤ 2014 → usa columna AM (NEDC o FTP-75)\nSi “Tecnología” = BEV (Battery Electric Vehicle) o FCEV → emisiones = 0\n\n'

# Limpieza datos NOMINA DE VEHICULOS

### llenar valores faltantes CO2


In [4]:
nomina_vehiculos = pd.read_excel("../utils/Nomina_vehiculos.xlsx")
nomina_vehiculos.head()

,id_modelo,marca,modelo,Transmisión,Combustible,Propulsión,Cilindrada,Carroceria,Emisiones de CO2 (g/km),CO2 combinado,Categoria_año,año_abajo,año_arriba
0,3299,Jaguar,"XF 2,2 Lts. DOHC Sedán 4P. T/A Motor Diésel",A,Diésel,Combustión,2.2,Sedán,139,-,2013-2025,2013,2025
1,3300,Jaguar,"XF 2,0T Lts. DOHC Sedán 4P. T/A Motor Otto",A,Gasolina,Combustión,2.0,Sedán,207,-,2013-2025,2013,2025
2,3301,Land Rover,"Freelander 2 2,0 Lts. DOHC Station Wagon 5P. 4...",A,Gasolina,Combustión,2.0,Station Wagon,224,-,2013-2025,2013,2025
3,3302,Piaggio,"Porter Maxxi Tipper Long Deck 1,3 Lts. DOHC Pi...",M,Gasolina,Combustión,1.3,Camioneta,227,-,2013-2025,2013,2025
4,NaN,Piaggio,"Porter Maxxi Chasis Cabina 1,3 Lts. DOHC Pick ...",M,Gasolina,Combustión,1.3,Camioneta,227,-,2013-2025,2013,2025


In [5]:
nomina_vehiculos_cols_nom = [normalizar_texto(col) for col in nomina_vehiculos.columns]
nomina_vehiculos.columns = nomina_vehiculos_cols_nom
nomina_vehiculos.head()

,id_modelo,marca,modelo,transmision,combustible,propulsion,cilindrada,carroceria,emisiones_de_co2_g_km,co2_combinado,categoria_ano,ano_abajo,ano_arriba
0,3299,Jaguar,"XF 2,2 Lts. DOHC Sedán 4P. T/A Motor Diésel",A,Diésel,Combustión,2.2,Sedán,139,-,2013-2025,2013,2025
1,3300,Jaguar,"XF 2,0T Lts. DOHC Sedán 4P. T/A Motor Otto",A,Gasolina,Combustión,2.0,Sedán,207,-,2013-2025,2013,2025
2,3301,Land Rover,"Freelander 2 2,0 Lts. DOHC Station Wagon 5P. 4...",A,Gasolina,Combustión,2.0,Station Wagon,224,-,2013-2025,2013,2025
3,3302,Piaggio,"Porter Maxxi Tipper Long Deck 1,3 Lts. DOHC Pi...",M,Gasolina,Combustión,1.3,Camioneta,227,-,2013-2025,2013,2025
4,NaN,Piaggio,"Porter Maxxi Chasis Cabina 1,3 Lts. DOHC Pick ...",M,Gasolina,Combustión,1.3,Camioneta,227,-,2013-2025,2013,2025


In [6]:
nomina_vehiculos.cilindrada.unique()

array([2.2 , 2.  , 1.3 , 1.6 , 1.2 , 2.5 , 1.  , 6.7 , 3.  , 3.5 , 3.2 ,
       1.8 , 4.7 , 4.  , 1.4 , 2.4 , 4.8 , 4.2 , 1.5 , 0.  , 5.  , 5.7 ,
       1.1 , 3.4 , 2.8 , 3.8 , 4.1 , 5.9 , 2.7 , 3.6 , 2.1 , 3.3 , 1.7 ,
       6.2 , 5.3 , 5.5 , 6.3 , 4.4 , 5.2 , 3.7 , 0.9 , 6.6 , 6.  , 0.8 ,
       2.3 , 4.6 , 3.9 , 5.01, 6.5 , 5.6 , 1.9 , 2.9 , 4.3 , 5.4 , 6.1 ,
       4.5 , 6.4 , 2.6 , 0.7 , 7.  , 0.97, 3.1 ])

In [7]:
nomina_vehiculos.propulsion.unique()

array(['Combustión', 'Vehículo eléctrico',
       'Vehículos híbridos sin recarga exterior',
       'Vehículos híbrido con recarga exterior',
       'Vehículos celda de hidrógeno', 'Eléctrico de Rango Extendido',
       'Eléctrico híbrido con recarga exterior', '-'], dtype=object)

In [8]:
# Limpiar valores no numéricos en las columnas de CO2
cols_co2 = ["emisiones_de_co2_g_km", "co2_combinado"]

for col in cols_co2:
    nomina_vehiculos[col] = (
        nomina_vehiculos[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace("-", "", regex=False)
        .str.replace("N/D", "", regex=False)
        .str.replace("n/a", "", regex=False)
        .str.strip()
    )

    nomina_vehiculos[col] = pd.to_numeric(nomina_vehiculos[col], errors="coerce")


In [9]:
nomina_vehiculos["es_electrico"] = (
    nomina_vehiculos["propulsion"]
    .astype(str)
    .str.lower()
    .str.contains("eléctrico")
)

In [28]:
vehiculos_df

,id_modelo,region,folio,tipo_servicio,ppu,fecha_ingreso_rnt,marca,modelo,ano_fabricacion,capacidad,año_archivo,mes_archivo,archivo,hoja,fecha_ingreso,combustible,linea,tipo_servicio_normalizado,marca_normalizada,modelo_normalizado
0,0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_gl_1_5
1,1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_1_6
2,2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6
3,3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6
4,4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480776,10480776,16.0,10036,AUTOMOVIL URBANO TAXI BÁSICO,BCSJ39,2008-02-13-00.00.00.000000,NISSAN,V16 EX SALOON 1.6,2008.0,4.0,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI BÁSICO,nissan,v16_ex_saloon_1_6
10480777,10480777,16.0,39,AUTOMOVIL URBANO TAXI COLECTIVO,FDXL32,2014-05-26-00.00.00.000000,TOYOTA,NEW YARIS SEDAN GLI 1.5,2014.0,4.0,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris_sedan_gli_1_5
10480778,10480778,16.0,94,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS66,2019-11-29-00.00.00.000000,PEUGEOT,301 ALLURE HDI,2020.0,NaN,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,peugeot,301_allure_hdi
10480779,10480779,16.0,127,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS65,2019-11-29-00.00.00.000000,TOYOTA,NEW YARIS,2020.0,NaN,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris


In [10]:
"""
-------------------------------
    VAMOS A AGREGAR LOS VALORES NULOS SACANDO EL PROMEDIO POR CILINDRADA Y TIPO DE CARROCERIA
-------------------------------

"""


nomina_vehiculos = nomina_vehiculos.copy()

# 1) Crear cilindrada agrupada
nomina_vehiculos["cilindrada_grp"] = nomina_vehiculos["cilindrada"].astype(float).round(1)

# ----------------------------------------------------------------------
# 2) Filtrar SOLO COMBUSTIÓN para calcular promedios
# ----------------------------------------------------------------------
nomina_vehiculos_comb = nomina_vehiculos[
    (nomina_vehiculos["propulsion"] != "Vehículo eléctrico") &
    (nomina_vehiculos["emisiones_de_co2_g_km"].notna())
]

# Promedio por carrocería + cilindrada + combustible
grp = (
    nomina_vehiculos_comb.groupby(["carroceria", "cilindrada_grp", "combustible"])["emisiones_de_co2_g_km"]
    .mean()
    .reset_index()
    .rename(columns={"emisiones_de_co2_g_km": "co2_prom_grupo"})
)

# Fallback por cilindrada
fallback_cil = (
    nomina_vehiculos_comb.groupby("cilindrada_grp")["emisiones_de_co2_g_km"]
    .mean()
    .reset_index()
    .rename(columns={"emisiones_de_co2_g_km": "co2_prom_cil"})
)

# Promedio global (solo combustión)
fallback_global = nomina_vehiculos_comb["emisiones_de_co2_g_km"].mean()

# ----------------------------------------------------------------------
# 3) Merge promedios SOLO a los vehículos de combustión
# ----------------------------------------------------------------------
nomina_vehiculos = nomina_vehiculos.merge(grp, on=["carroceria","cilindrada_grp","combustible"], how="left")
nomina_vehiculos = nomina_vehiculos.merge(fallback_cil, on="cilindrada_grp", how="left")







In [11]:

# ----------------------------------------------------------------------
# 4) Función final para calcular CO2
# ----------------------------------------------------------------------
def calcular_co2(row):

    # Caso especial: eléctrico → SIEMPRE 0 (sin usar promedios)
    if row["es_electrico"]:
        return 0


    # Valores crudos
    c1 = row["emisiones_de_co2_g_km"]
    c2 = row["co2_combinado"]

    # Caso 1: ambos presentes → promedio
    if pd.notna(c1) and pd.notna(c2):
        return (c1 + c2) / 2

    # Caso 2: solo co2_combinado
    if pd.isna(c1) and pd.notna(c2):
        return c2

    # Caso 3: solo emisiones
    if pd.notna(c1):
        return c1

    # Caso 4: ambos vacíos → imputación SOLO si combustión
    if row["propulsion"] == "Combustión":
        
        if pd.notna(row["co2_prom_grupo"]):
            return row["co2_prom_grupo"]

        if pd.notna(row["co2_prom_cil"]):
            return row["co2_prom_cil"]

        return fallback_global
    
    # Si no es combustión y no tiene datos → se queda NaN
    return np.nan

In [12]:
nomina_vehiculos["co2_final"] = nomina_vehiculos.apply(calcular_co2, axis=1)

print("IMPUTACIÓN LISTA ✔ Vehículos eléctricos excluidos correctamente.")

IMPUTACIÓN LISTA ✔ Vehículos eléctricos excluidos correctamente.


In [13]:
nomina_vehiculos

,id_modelo,marca,modelo,transmision,combustible,propulsion,cilindrada,carroceria,emisiones_de_co2_g_km,co2_combinado,categoria_ano,ano_abajo,ano_arriba,es_electrico,cilindrada_grp,co2_prom_grupo,co2_prom_cil,co2_final
0,3299,Jaguar,"XF 2,2 Lts. DOHC Sedán 4P. T/A Motor Diésel",A,Diésel,Combustión,2.2,Sedán,139.0,NaN,2013-2025,2013,2025,False,2.2,152.333333,215.258413,139.0
1,3300,Jaguar,"XF 2,0T Lts. DOHC Sedán 4P. T/A Motor Otto",A,Gasolina,Combustión,2.0,Sedán,207.0,NaN,2013-2025,2013,2025,False,2.0,181.324607,182.270695,207.0
2,3301,Land Rover,"Freelander 2 2,0 Lts. DOHC Station Wagon 5P. 4...",A,Gasolina,Combustión,2.0,Station Wagon,224.0,NaN,2013-2025,2013,2025,False,2.0,191.571053,182.270695,224.0
3,3302,Piaggio,"Porter Maxxi Tipper Long Deck 1,3 Lts. DOHC Pi...",M,Gasolina,Combustión,1.3,Camioneta,227.0,NaN,2013-2025,2013,2025,False,1.3,187.452830,173.140000,227.0
4,NaN,Piaggio,"Porter Maxxi Chasis Cabina 1,3 Lts. DOHC Pick ...",M,Gasolina,Combustión,1.3,Camioneta,227.0,NaN,2013-2025,2013,2025,False,1.3,187.452830,173.140000,227.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13068,985,Subaru,"Impreza 1,6 AWD SOHC Sedán 4P. T/A Motor Otto",A,Gasolina,Combustión,1.6,Sedán,187.0,NaN,1997-2002,1997,2002,False,1.6,173.517787,169.256000,187.0
13069,986,Volkswagen,"Caddy 1,6 SOHC Furgón 3P. T/M Motor Otto Dual",M,Gasolina/GNC,Combustión,1.6,Furgón,187.0,NaN,1997-2002,1997,2002,False,1.6,187.000000,169.256000,187.0
13070,987,Citroen,"C3 1,4 SOHC Hatchback 5P. T/M Motor Otto",M,Gasolina,Combustión,1.4,Hatchback,187.0,NaN,1997-2002,1997,2002,False,1.4,153.463636,155.202733,187.0
13071,988,Kia,"Sorento 3,5 DOHC Tipo Jeep 4P. AWD T/A Motor Otto",A,Gasolina,Combustión,3.5,Station Wagon,187.0,NaN,1997-2002,1997,2002,False,3.5,244.258824,239.136646,187.0


# Region


In [14]:
# USARÉ EL EXCEL MAS RECIENTE PARA COMPRAR LA COLUMNA CON EL ID DE REGION
filepath_transpublico_oct = r"C:\Users\hiros\Desktop\CMS-PRUEBA-TECNICA\descargas\2025\Transporte Público - Octubre 2025.xlsx"

excel = pd.ExcelFile(filepath_transpublico_oct)
excel.sheet_names


['Servicios', 'RepLegales', 'Vehículos', 'O-D Recorridos', 'Trazados']

In [15]:
excel_servicios = excel.parse(sheet_name="Servicios")
excel_servicios

,REGION,FOLIO,TIPO_SERVICIO,FLOTA _VIGENTE,NRO_LINEA,RUT_RESPONSABLE,NOMBRE_RESPONSABLE,NOMBRE_FANTASIA,TIPO_PERSONA,REGION.1,COMUNA,DOMICILIO,TELEFONO,EMAIL
0,1,2367,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,NaN,PATRICIA H. MANCILLA PEREIRA,NaN,NATURAL,de Tarapacá,NaN,NaN,NaN,NaN
1,1,4041,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,NaN,PATRICIA H. MANCILLA PEREIRA,NaN,NATURAL,de Tarapacá,NaN,NaN,NaN,NaN
2,1,3850,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,NaN,ALEJANDRO BENJAMÍN DURANDAL CASTRO,NaN,NATURAL,de Tarapacá,NaN,NaN,NaN,NaN
3,1,1334,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,NaN,PEDRO ARTEMIO CARRILLO MU?OZ,NaN,NATURAL,de Tarapacá,NaN,NaN,NaN,NaN
4,1,1568,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,NaN,CARLOS MAURICIO DEL CARMEN VALLE CASTILLO,NaN,NATURAL,de Tarapacá,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39751,16,10260,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,76978141-2,NAVA TAXI LIMITADA,NAVA TAXI,JURIDICA,de ?uble,Coihueco,COMERCIO 1320,932573886,navaempresa@gmail.com
39752,16,10289,AUTOMOVIL URBANO TAXI BﾁSICO,1,NaN,76978141-2,NAVA TAXI LIMITADA,NAVA TAXI,JURIDICA,de ?uble,Coihueco,COMERCIO 1320,932573886,navaempresa@gmail.com
39753,16,12021,AUTOMOVIL URBANO TAXI TURISMO,1,NaN,76978141-2,NAVA TAXI LIMITADA,NAVA TAXI,JURIDICA,de ?uble,Coihueco,COMERCIO 1320,932573886,navaempresa@gmail.com
39754,16,12210,AUTOMOVIL URBANO TAXI TURISMO,1,NaN,76978141-2,NAVA TAXI LIMITADA,NAVA TAXI,JURIDICA,de ?uble,Coihueco,COMERCIO 1320,932573886,navaempresa@gmail.com


In [16]:
regiones = excel_servicios[['REGION','REGION.1']]
regiones.drop_duplicates(inplace=True)
regiones.rename(columns={'REGION':'id_region','REGION.1':'region'},inplace=True)
regiones

C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\2976192624.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regiones.drop_duplicates(inplace=True)
C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\2976192624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regiones.rename(columns={'REGION':'id_region','REGION.1':'region'},inplace=True)


,id_region,region
0,1,de Tarapacá
3816,1,de Antofagasta
3817,1,de Valparaíso
3819,1,del Maule
3820,1,del Biobío
...,...,...
38121,15,de Arica y Parinacota
38749,16,del Biobío
38756,16,Metropolitana de Santiago
38757,16,de ?uble


# DATAFRAME VEHICULOS


In [17]:
vehiculos_filepath = r"C:\Users\hiros\Desktop\CMS-PRUEBA-TECNICA\Normalizar datos\vehiculos_extraidos.csv"
vehiculos_df = pd.read_csv(vehiculos_filepath)
vehiculos_df.rename(columns={'Unnamed: 0':'id_modelo'},inplace=True)
vehiculos_df

C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\996306710.py:2: DtypeWarning: Columns (5,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  vehiculos_df = pd.read_csv(vehiculos_filepath)


,id_modelo,region,folio,tipo_servicio,ppu,fecha_ingreso_rnt,marca,modelo,ano_fabricacion,capacidad,año_archivo,mes_archivo,archivo,hoja,fecha_ingreso,combustible,linea
0,0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN
1,1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN
2,2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN
3,3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN
4,4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,2021,abril,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10480787,10480787,16.0,500700,BUS RURAL CORRIENTE,DKWJ44,2013-06-03-15.58.03.000000,SCANIA,K380B,2012.0,44.0,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN
10480788,10480788,16.0,500936,BUS RURAL CORRIENTE,BXDK27,2015-11-10-17.26.01.000000,MERCEDES BENZ,0500 RS,2009.0,NaN,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN
10480789,10480789,16.0,500825,BUS RURAL CORRIENTE,KXBT57,2018-11-09-00.00.00.000000,MERCEDES BENZ,OF 1724,2019.0,46.0,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN
10480790,10480790,16.0,400080,BUS URBANO CORRIENTE,WK7479,2007-08-10-12.00.00.000000,MERCEDES BENZ,LO 914 42,2007.0,NaN,2019,diciembre,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN


In [18]:
vehiculos_df['tipo_servicio'].unique()

array(['TAXI COLECTIVO RURAL', 'TAXI BASICO', 'TAXI COLECTIVO URBANO',
       'TAXI EJECUTIVO', 'TAXI TURISMO', 'BUS AEROPUERTO RECORRIDO FIJO',
       'BUS AEROPUERTO RECORRIDO VARIABLE', 'BUS INTERURBANO CORRIENTE',
       'BUS RURAL CORRIENTE', 'BUS RURAL PERIFERICO',
       'BUS URBANO CORRIENTE', 'BUS URBANO TRANSANTIAGO',
       'MINIBUS AEROPUERTO RECORRIDO VARIABLE', 'MINIBUS RURAL CORRIENTE',
       'MINIBUS URBANO CORRIENTE', 'AUTOMOVIL RURAL TAXI COLECTIVO',
       'AUTOMOVIL URBANO TAXI COLECTIVO', 'AUTOMOVIL URBANO TAXI TURISMO',
       'AUTOMOVIL URBANO TAXI BﾁSICO', 'AUTOMOVIL URBANO TAXI EJECUTIVO',
       'BUS RURAL PERIFﾉRICO', 'AUTOMOVIL URBANO TAXI BASICO',
       'MINIBUS AEROPUERTO RECORRIDO FIJO',
       'STATIONWAGON AEROPUERTO RECORRIDO VARIABLE',
       'AUTOMOVIL RURAL TAXICOLECTIVO', 'AUTOMOVIL URBANO TAXICOLECTIVO',
       'AUTOMOVIL URBANO TAXITURISMO', 'AUTOMOVIL URBANO TAXIBASICO',
       'AUTOMOVIL URBANO TAXIEJECUTIVO',
       'AUTOMOVIL URBANOLICITADO

In [19]:
# diccionario 

normalizacion_tipo_servicio = {

    # --------------------
    # TAXI - BASICO
    # --------------------
    "TAXI BASICO": "TAXI BÁSICO",
    "AUTOMOVIL URBANO TAXI BASICO": "TAXI BÁSICO",
    "AUTOMOVIL URBANO TAXIBASICO": "TAXI BÁSICO",
    "AUTOMOVIL URBANOLICITADO TAXIBASICO": "TAXI BÁSICO",
    "AUTOMOVIL TAXIBASICO URBANO": "TAXI BÁSICO",
    "AUTOMOVIL URBANO TAXI BﾁSICO": "TAXI BÁSICO",
    "AUTOMOVIL URBANO TAXI BÁSICO": "TAXI BÁSICO",

    # --------------------
    # TAXI - EJECUTIVO
    # --------------------
    "TAXI EJECUTIVO": "TAXI EJECUTIVO",
    "AUTOMOVIL URBANO TAXI EJECUTIVO": "TAXI EJECUTIVO",
    "AUTOMOVIL URBANO TAXIEJECUTIVO": "TAXI EJECUTIVO",
    "AUTOMOVIL URBANOLICITADO TAXIEJECUTIVO": "TAXI EJECUTIVO",
    "AUTOMOVIL TAXIEJECUTIVO URBANO": "TAXI EJECUTIVO",

    # --------------------
    # TAXI - TURISMO
    # --------------------
    "TAXI TURISMO": "TAXI TURISMO",
    "AUTOMOVIL URBANO TAXI TURISMO": "TAXI TURISMO",
    "AUTOMOVIL URBANO TAXITURISMO": "TAXI TURISMO",
    "AUTOMOVIL URBANOLICITADO TAXITURISMO": "TAXI TURISMO",
    "AUTOMOVIL TAXITURISMO URBANO": "TAXI TURISMO",

    # --------------------
    # TAXI COLECTIVO
    # --------------------
    "TAXI COLECTIVO RURAL": "TAXI COLECTIVO RURAL",
    "AUTOMOVIL RURAL TAXI COLECTIVO": "TAXI COLECTIVO RURAL",
    "AUTOMOVIL RURAL TAXICOLECTIVO": "TAXI COLECTIVO RURAL",
    "AUTOMOVIL TAXICOLECTIVO RURAL": "TAXI COLECTIVO RURAL",

    "TAXI COLECTIVO URBANO": "TAXI COLECTIVO URBANO",
    "AUTOMOVIL URBANO TAXI COLECTIVO": "TAXI COLECTIVO URBANO",
    "AUTOMOVIL URBANO TAXICOLECTIVO": "TAXI COLECTIVO URBANO",
    "AUTOMOVIL TAXICOLECTIVO URBANO": "TAXI COLECTIVO URBANO",
    "AUTOMOVIL TAXICOLECTIVO URBANOLICITADO": "TAXI COLECTIVO URBANO",
    "AUTOMOVIL URBANOLICITADO TAXICOLECTIVO": "TAXI COLECTIVO URBANO",

    # --------------------
    # BUS - AEROPUERTO
    # --------------------
    "BUS AEROPUERTO RECORRIDO FIJO": "BUS AEROPUERTO RECORRIDO FIJO",
    "BUS AEROPUERTO RECORRIDOFIJO": "BUS AEROPUERTO RECORRIDO FIJO",
    "BUS RECORRIDOFIJO AEROPUERTO": "BUS AEROPUERTO RECORRIDO FIJO",

    "BUS AEROPUERTO RECORRIDO VARIABLE": "BUS AEROPUERTO RECORRIDO VARIABLE",
    "BUS AEROPUERTO RECORRIDOVARIABLE": "BUS AEROPUERTO RECORRIDO VARIABLE",
    "BUS RECORRIDOVARIABLE AEROPUERTO": "BUS AEROPUERTO RECORRIDO VARIABLE",

    # --------------------
    # MINIBUS - AEROPUERTO
    # --------------------
    "MINIBUS AEROPUERTO RECORRIDO FIJO": "MINIBUS AEROPUERTO RECORRIDO FIJO",
    "MINIBUS AEROPUERTO RECORRIDOFIJO": "MINIBUS AEROPUERTO RECORRIDO FIJO",
    "MINIBUS RECORRIDOFIJO AEROPUERTO": "MINIBUS AEROPUERTO RECORRIDO FIJO",

    "MINIBUS AEROPUERTO RECORRIDO VARIABLE": "MINIBUS AEROPUERTO RECORRIDO VARIABLE",
    "MINIBUS AEROPUERTO RECORRIDOVARIABLE": "MINIBUS AEROPUERTO RECORRIDO VARIABLE",
    "MINIBUS RECORRIDOVARIABLE AEROPUERTO": "MINIBUS AEROPUERTO RECORRIDO VARIABLE",

    # --------------------
    # BUS - CORRIENTE Varios
    # --------------------
    "BUS INTERURBANO CORRIENTE": "BUS CORRIENTE INTERURBANO",
    "BUS CORRIENTE INTERURBANO": "BUS CORRIENTE INTERURBANO",

    "BUS RURAL CORRIENTE": "BUS CORRIENTE RURAL",
    "BUS CORRIENTE RURAL": "BUS CORRIENTE RURAL",
    "BUS RURALLICITADO CORRIENTE": "BUS CORRIENTE RURAL",
    "BUS CORRIENTE RURALLICITADO": "BUS CORRIENTE RURAL",

    "BUS RURAL PERIFERICO": "BUS CORRIENTE PERIFERICO",
    "BUS RURAL PERIFÉRICO": "BUS CORRIENTE PERIFERICO",
    "BUS CORRIENTE PERIFERICO": "BUS CORRIENTE PERIFERICO",

    "BUS URBANO CORRIENTE": "BUS CORRIENTE URBANO",
    "BUS CORRIENTE URBANO": "BUS CORRIENTE URBANO",
    "BUS URBANONOLICITADO CORRIENTE": "BUS CORRIENTE URBANO",
    "BUS CORRIENTE URBANONOLICITADO": "BUS CORRIENTE URBANO",

    "BUS URBANOLICITADO CORRIENTE": "BUS CORRIENTE URBANOLICITADO",
    "BUS CORRIENTE URBANOLICITADO": "BUS CORRIENTE URBANOLICITADO",

    "BUS URBANOLICITADO TRANSANTIAGO": "BUS TRANSANTIAGO",
    "BUS TRANSANTIAGO URBANOLICITADO": "BUS TRANSANTIAGO",

    "BUS URBANO TRANSANTIAGO": "BUS TRANSANTIAGO",

    # --------------------
    # ESCOLARES
    # --------------------
    "BUS URBANO ESCOLAR": "BUS ESCOLAR",
    "BUS RURAL ESCOLAR": "BUS ESCOLAR",

    "MINIBUS URBANO ESCOLAR": "MINIBUS ESCOLAR",
    "MINIBUS RURAL ESCOLAR": "MINIBUS ESCOLAR",
}


In [20]:
vehiculos_df["tipo_servicio_normalizado"] = (
    vehiculos_df["tipo_servicio"].map(normalizacion_tipo_servicio)
)


In [21]:
# que contenga la plabra taxi
vehiculos_df = vehiculos_df[vehiculos_df['tipo_servicio_normalizado'].str.contains('TAXI', na=False)]

In [22]:
normalizacion = {
    # HYUNDAI
    'hyundai':'hyundai','huindai':'hyundai','hyunday':'hyundai','hyunndai':'hyundai',
    'hyndai':'hyundai','hundai':'hyundai','hyn dai':'hyundai','hyuundai':'hyundai',
    'yhundai':'hyundai','huyndai':'hyundai','hiunday':'hyundai','yundai':'hyundai',
    'hiundai':'hyundai','huyundai':'hyundai','hyubdai':'hyundai','hyunda':'hyundai',
    'hyunjdai':'hyundai','hyunnda':'hyundai',

    # KIA
    'kia':'kia','kia_motors':'kia','kia motors':'kia','kia_motor':'kia',
    'kia_motos':'kia','kiaa':'kia','kia_soluto':'kia','k5':'kia',

    # PEUGEOT
    'peugeot':'peugeot','peogeot':'peugeot','peugeout':'peugeot',

    # CHEVROLET
    'chevrolet':'chevrolet','chevrlet':'chevrolet','chevrolt':'chevrolet',

    # TOYOTA
    'toyota':'toyota','tpoyota':'toyota','toyoa':'toyota','toyoya':'toyota',
    'toyotq':'toyota','tpypta':'toyota','totyota':'toyota','toyotas':'toyota',
    'toyoia':'toyota','toyoto':'toyota',

    # NISSAN
    'nissan':'nissan',

    # CITROEN
    'citroen':'citroen','citroe':'citroen','citreon':'citroen','cytroen':'citroen',
    'c_elysee':'citroen','elysee':'citroen',

    # GREAT WALL
    'great_wall':'great_wall','graet_wall':'great_wall','great':'great_wall',
    'great_wall_motors':'great_wall','grat_wall':'great_wall',

    # SAMSUNG / RENAULT-SAMSUNG
    'samsung':'samsung','renault_samsung':'samsung',

    # SUZUKI
    'suzuki':'suzuki',

    # RENAULT
    'renault':'renault',

    # MAZDA
    'mazda':'mazda',

    # HONDA
    'honda':'honda',

    # BYD
    'byd':'byd','b_y_d':'byd',

    # HAIMA
    'haima':'haima',

    # VOLKSWAGEN
    'volkswagen':'volkswagen','volvagen':'volkswagen','volskwagen':'volkswagen',
    'volswagen':'volkswagen','volkagswaven':'volkswagen','volswagwn':'volkswagen',
    'wolkswagen':'volkswagen','volkswaven':'volkswagen','volkswagen ':'volkswagen',
    'volkag swagen':'volkswagen',

    # DFM / DONGFENG
    'dfm':'dongfeng','dflm':'dongfeng','dong_feng':'dongfeng','dongfeng':'dongfeng',

    # LIFAN
    'lifan':'lifan',

    # BRILLIANCE
    'brilliance':'brilliance',

    # MG
    'mg':'mg',

    # CHERY
    'chery':'chery','cherry':'chery',

    # JAC
    'jac':'jac','jac_motors':'jac',

    # GEELY
    'geely':'geely',

    # FORD
    'ford':'ford',

    # CHANGAN
    'changan':'changan','chnagan':'changan','chang an':'changan',

    # FAW
    'faw':'faw',

    # MITSUBISHI
    'mitsubishi':'mitsubishi','mitsubichi':'mitsubishi','mitsubhisi':'mitsubishi',
    'mithsubisgi':'mitsubishi','mitsubichi_fuso':'mitsubishi_fuso',
    'mitsubshi_fuso':'mitsubishi_fuso','mitsubishi_fusu':'mitsubishi_fuso',
    'mtsubishi_fuso':'mitsubishi_fuso','misubishi_fuso':'mitsubishi_fuso',
    'mitubishi_fuso':'mitsubishi_fuso','mitsu bishii':'mitsubishi',
    'mitsubishi_fuso':'mitsubishi_fuso','mitsubishi_fudu':'mitsubishi_fuso',
    'mitsubishi_rosa':'mitsubishi',

    # SMA
    'sma':'sma',

    # FIAT / DODGE
    'fiat':'fiat','dodge':'dodge',

    # VOLVO
    'volvo':'volvo','volovo':'volvo','vovlo':'volvo',

    # SUBARU
    'subaru':'subaru',

    # OPEL
    'opel':'opel',

    # MERCEDES-BENZ
    'mercedes_benz':'mercedes_benz','mercedes':'mercedes_benz','m_benz':'mercedes_benz',
    'mercedez':'mercedes_benz','mercedez_benz':'mercedes_benz',
    'mercedes_b':'mercedes_benz','m ben z':'mercedes_benz','mercedes_menz':'mercedes_benz',
    'm.benz':'mercedes_benz','m_benz':'mercedes_benz','mercedes_benez':'mercedes_benz',
    'mercede_benz':'mercedes_benz','mercedez_benz ':'mercedes_benz',
    'mercedes_b ':'mercedes_benz',

    # LADA
    'lada':'lada',

    # SKODA
    'skoda':'skoda',

    # PROTON
    'proton':'proton',

    # BMW
    'bmw':'bmw',

    # AUDI
    'audi':'audi',

    # DAEWOO
    'daewoo':'daewoo',

    # KINGLONG
    'kinglong':'kinglong','king_long':'kinglong',

    # SCANIA
    'scania':'scania','scani':'scania',

    # YUTONG / ZHONGTONG
    'yutong':'yutong','youtong':'yutong',
    'zhongtong':'zhongtong','zhong_tong':'zhongtong','zhongton':'zhongtong',
    'zhongtng':'zhongtong','zhon':'zhongtong','zhontong':'zhongtong',
    'zhonglong':'zhongtong','zhomgtong':'zhongtong','zhongton':'zhongtong',

    # HAFEI
    'hafei':'hafei',

    # YOUNGMAN
    'youngman':'youngman',

    # HIGER
    'higer':'higer',

    # GOLDEN DRAGON
    'golden_dragon':'golden_dragon','sun_long':'golden_dragon',

    # EUROSEC
    'eurosec':'eurosec',

    # MAN
    'man':'man',

    # MARCOPOLO
    'marcopolo':'marcopolo',

    # BONLUCK
    'bonluck':'bonluck',

    # SSANGYONG
    'ssangyong':'ssangyong',

    # ASHOK LEYLAND
    'ashok_leyland':'ashok_leyland',

    # IVECO
    'iveco':'iveco',

    # VOLARE
    'volare':'volare',

    # AGRALE
    'agrale':'agrale','agrale_walk':'agrale','agrale_irizar':'agrale',

    # FUSO
    'fuso':'fuso',

    # COMIL
    'comil':'comil',

    # YOUYI
    'youyi':'youyi',

    # SUNLONG
    'sunlong':'sunlong',

    # HUANGHAI
    'huanghai':'huanghai',

    # HENGTONG
    'hengtong':'hengtong',

    # INDABRA
    'indabra':'indabra',

    # YUEJIN
    'yuejin':'yuejin',

    # MAXUS
    'maxus':'maxus',

    # MUDAN
    'mudan':'mudan',

    # EXPRESS INTL
    'xpress_international':'xpress_international',

    # ASIA MOTORS
    'asia_motors':'asia_motors','asia':'asia_motors',

    # DIMEX
    'dimex':'dimex',

    # FTBCI
    'ftbci':'ftbci',

    # YAXING
    'yaxing':'yaxing',

    # MASA
    'masa':'masa',

    # ZONDA
    'zonda':'zonda',

    # NEOBUS
    'neobus':'neobus',

    # GEO
    'geo':'geo',

    # HINO
    'hino':'hino',

    # PULLMANN STANDARD
    'pullmann_standard':'pullmann_standard','pullman':'pullmann_standard',

    # NAW
    'naw':'naw',

    # HUAXIN
    'huaxin':'huaxin',

    # SHIYAN
    'shiyan':'shiyan',

    # FOTON
    'foton':'foton',

    # WRIGHTBUS
    'wrightbus':'wrightbus',

    # JINBEI
    'jinbei':'jinbei',

    # KINGSTAR
    'kingstar_pluto_j6':'kingstar_pluto_j6',

    # AVANTE
    'avante':'avante',

    # GAC
    'gac':'gac',

    # HAVAL
    'haval':'haval',

    # MAPLE
    'maple':'maple',

    # JMC
    'jmc':'jmc',

    # DFSK
    'dfsk':'dfsk',

    # BAIC
    'baic':'baic',

    # JETOUR
    'jetour':'jetour',

    # INFINITI
    'infiniti':'infiniti',

    # TESLA
    'tesla':'tesla',

    # OMODA
    'omoda':'omoda',

    # DS
    'ds':'ds',

    # LEXUS
    'lexus':'lexus',

    # TATA
    'tata':'tata',

    # JEEP
    'jeep':'jeep',

    # ISUZU
    'isuzu':'isuzu',

    # DAF
    'daf':'daf',

    # SEAT
    'seat':'seat',

    # NAVISTAR
    'navistar':'navistar',

    # KOMATSU
    'komatsu':'komatsu',

    # SINOTRUK
    'sinotruk':'sinotruk',

    # FREIGHTLINER
    'freightliner':'freightliner',
}


In [23]:

vehiculos_df['marca_normalizada'] = vehiculos_df['marca'].apply(normalizar_texto)

vehiculos_df["marca_normalizada"] = (
    vehiculos_df["marca_normalizada"].map(normalizacion)
)


C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\773138803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehiculos_df['marca_normalizada'] = vehiculos_df['marca'].apply(normalizar_texto)
C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\773138803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehiculos_df["marca_normalizada"] = (


In [24]:
vehiculos_df['modelo_normalizado'] = vehiculos_df['modelo'].apply(normalizar_texto)

C:\Users\hiros\AppData\Local\Temp\ipykernel_6908\2839402485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehiculos_df['modelo_normalizado'] = vehiculos_df['modelo'].apply(normalizar_texto)


In [25]:
nomina_vehiculos["marca_normalizada"] = (
    nomina_vehiculos["marca"].apply(normalizar_texto)
)
nomina_vehiculos['marca_normalizada'] = nomina_vehiculos['marca_normalizada'].map(normalizacion)




In [26]:

nomina_vehiculos["modelo_normalizado"] = (
    nomina_vehiculos["modelo"].apply(normalizar_texto)
)

In [27]:
asdasd

NameError: name 'asdasd' is not defined

# Homologación de vehiculos

In [ ]:
import pandas as pd
from rapidfuzz import fuzz, process

# ============================================================
# 1) Determinar el modelo que mejor representa a un grupo
# ============================================================
def modelo_representativo(modelos):
    """
    Para un conjunto de nombres de modelos que pertenecen al mismo id_modelo,
    devuelve el que más se parece al resto del grupo (el 'centroide').
    """
    modelos = list(modelos)

    if len(modelos) == 1:
        return modelos[0]

    # Calcula el puntaje promedio de similitud entre cada modelo y los demás
    scores = []
    for m in modelos:
        s = [fuzz.token_sort_ratio(m, n) for n in modelos]
        scores.append(sum(s) / len(s))

    # Devuelve el modelo con mayor puntaje promedio
    return modelos[scores.index(max(scores))]

# ============================================================
# 2) Agrupa la nómina por id_modelo y marca
# ============================================================
grupos_nomina = (
    nomina_vehiculos
    .groupby(["id_modelo", "marca_normalizada"], as_index=False)
    .agg({"modelo_normalizado": modelo_representativo})
)

# ============================================================
# 3) Crea un diccionario de modelos para cada marca
# ============================================================
modelos_tp_dict = (
    vehiculos_df
    .groupby("marca_normalizada")["modelo_normalizado"]
    .apply(lambda x: list(x.unique()))
    .to_dict()
)

# ============================================================
# 4) Encuentra el mejor match de un modelo dentro de la misma marca
# ============================================================
def mejor_match(marca, modelo_base):
    if marca not in modelos_tp_dict:
        return None, 0

    opciones = modelos_tp_dict[marca]

    match, score, _ = process.extractOne(
        modelo_base,
        opciones,
        scorer=fuzz.token_sort_ratio
    )

    return match, score

# ============================================================
# 5) Genera los matches
# ============================================================
resultados = []

for _, row in grupos_nomina.iterrows():
    id_mod = row["id_modelo"]
    marca = row["marca_normalizada"]
    modelo = row["modelo_normalizado"]

    match, score = mejor_match(marca, modelo)

    resultados.append({
        "id_modelo": id_mod,
        "marca_normalizada": marca,
        "modelo_representativo": modelo,
        "modelo_tp_match": match,
        "score": score
    })

matches_df = pd.DataFrame(resultados)


In [ ]:
matches_df

,id_modelo,marca_normalizada,modelo_representativo,modelo_tp_match,score
0,1,peugeot,306_1_4_lts_sedan_4p_t_m,301_vti_1_6_aut,46.153846
1,2,peugeot,306_1_6_lts_sedan_4p_t_m_motor_nfz,301_active_pack_vti_1_6_aut,42.622951
2,3,honda,civic_1_6_lts_sedan_ex_4p_t_m_motor_otto,civic_exlt_1_5_aut,44.827586
3,4,honda,civic_sedan_lx_4p_1_6_lts_t_m,civic_ex_1_5_at,59.090909
4,5,honda,accord_ex_sedan_4p_2_2_lts_t_a,accord_ex_l_2_4_aut,61.224490
...,...,...,...,...,...
6470,7521,volkswagen,amarok_pa2_v6,bora_a4_europa_2_0,51.612903
6471,7522,dongfeng,t5_1_6l,t5_1_6,92.307692
6472,7523,foton,tm5_cargobox,None,0.000000
6473,7542,volvo,xc90_t8_awd_core,s60_d2_comfort,46.666667


In [ ]:


matches_final = matches_df[matches_df["score"] >= 60].copy()


print("=== MATCHES FINALES ===")
matches_final


=== MATCHES FINALES ===


,id_modelo,marca_normalizada,modelo_representativo,modelo_tp_match,score
4,5,honda,accord_ex_sedan_4p_2_2_lts_t_a,accord_ex_l_2_4_aut,61.224490
11,12,honda,accord_ex_r_2_2_lts_t_a,accord_ex_l_2_4_aut,71.428571
29,30,ford,mondeo_clx_1_8_lts_t_a,mondeo_clx_1_8,77.777778
30,31,ford,mondeo_clx_1_8_lts_t_m,mondeo_clx_1_8,77.777778
40,41,ford,mondeo_glx_2_0_lts_t_a,mondeo_clx_1_8,61.111111
...,...,...,...,...,...
6460,7505,dongfeng,mage_1_5t,mage_1_5_at,90.000000
6464,7512,volkswagen,polo_1_6_mpi,polo_1_6,80.000000
6467,7516,jetour,x50_1_5_mt,x70_gl_1_5,60.000000
6468,7519,mg,mg_hs_1_5t_dct,mg_zs_1_5,69.565217


In [ ]:
matches_final = matches_final.merge(nomina_vehiculos[['id_modelo','combustible']], on='id_modelo', how='left')
matches_final

,id_modelo,marca_normalizada,modelo_representativo,modelo_tp_match,score,combustible
0,5,honda,accord_ex_sedan_4p_2_2_lts_t_a,accord_ex_l_2_4_aut,61.224490,Gasolina
1,12,honda,accord_ex_r_2_2_lts_t_a,accord_ex_l_2_4_aut,71.428571,Gasolina
2,30,ford,mondeo_clx_1_8_lts_t_a,mondeo_clx_1_8,77.777778,Gasolina
3,31,ford,mondeo_clx_1_8_lts_t_m,mondeo_clx_1_8,77.777778,Gasolina
4,41,ford,mondeo_glx_2_0_lts_t_a,mondeo_clx_1_8,61.111111,Gasolina
...,...,...,...,...,...,...
791,7505,dongfeng,mage_1_5t,mage_1_5_at,90.000000,Gasolina
792,7512,volkswagen,polo_1_6_mpi,polo_1_6,80.000000,Gasolina
793,7516,jetour,x50_1_5_mt,x70_gl_1_5,60.000000,Gasolina
794,7519,mg,mg_hs_1_5t_dct,mg_zs_1_5,69.565217,Gasolina


In [ ]:
vehiculos_df.combustible.isnull().sum()

6033482

In [ ]:
# Merge de los matches con el vehiculo
vehiculos = vehiculos_df.merge(
    matches_final[['modelo_tp_match','combustible']], left_on='modelo_normalizado', right_on='modelo_tp_match', how='left')
vehiculos

,Unnamed: 0,region,folio,tipo_servicio,ppu,fecha_ingreso_rnt,marca,modelo,ano_fabricacion,capacidad,...,archivo,hoja,fecha_ingreso,combustible_x,linea,tipo_servicio_normalizado,marca_normalizada,modelo_normalizado,modelo_tp_match,combustible_y
0,0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,...,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_gl_1_5,NaN,NaN
1,1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,...,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_1_6,NaN,NaN
2,2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,...,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN
3,3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,...,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN
4,4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,...,Base de Datos de Transporte Público - Abril 20...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7827195,10480776,16.0,10036,AUTOMOVIL URBANO TAXI BÁSICO,BCSJ39,2008-02-13-00.00.00.000000,NISSAN,V16 EX SALOON 1.6,2008.0,4.0,...,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI BÁSICO,nissan,v16_ex_saloon_1_6,NaN,NaN
7827196,10480777,16.0,39,AUTOMOVIL URBANO TAXI COLECTIVO,FDXL32,2014-05-26-00.00.00.000000,TOYOTA,NEW YARIS SEDAN GLI 1.5,2014.0,4.0,...,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris_sedan_gli_1_5,NaN,NaN
7827197,10480778,16.0,94,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS66,2019-11-29-00.00.00.000000,PEUGEOT,301 ALLURE HDI,2020.0,NaN,...,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,peugeot,301_allure_hdi,NaN,NaN
7827198,10480779,16.0,127,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS65,2019-11-29-00.00.00.000000,TOYOTA,NEW YARIS,2020.0,NaN,...,Base de Datos de Transporte Público - Diciembr...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris,NaN,NaN


In [ ]:
# juntar combustible_x y _x en una sola sola... si el valor es nulo en :y se mantiene el de :x si es que hay un valor en _y re remplaza el de x
vehiculos['combustible_final'] = vehiculos['combustible_x'].combine_first(vehiculos['combustible_y'])
vehiculos

,Unnamed: 0,region,folio,tipo_servicio,ppu,fecha_ingreso_rnt,marca,modelo,ano_fabricacion,capacidad,...,hoja,fecha_ingreso,combustible_x,linea,tipo_servicio_normalizado,marca_normalizada,modelo_normalizado,modelo_tp_match,combustible_y,combustible_final
0,0,1.0,10,TAXI COLECTIVO RURAL,RW2882,2007-06-26,HYUNDAI,ACCENT GL 1.5,2007.0,4.0,...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_gl_1_5,NaN,NaN,NaN
1,1,1.0,12,TAXI COLECTIVO RURAL,BCRH67,2019-10-07,HYUNDAI,ACCENT 1.6,2015.0,NaN,...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_1_6,NaN,NaN,NaN
2,2,1.0,18,TAXI COLECTIVO RURAL,DWCY37,2012-05-15,HYUNDAI,ACCENT RB GL 1.6,2012.0,4.0,...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN,NaN
3,3,1.0,19,TAXI COLECTIVO RURAL,DGFX88,2011-08-30,HYUNDAI,ACCENT RB GL 1.6,2011.0,4.0,...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN,NaN
4,4,1.0,24,TAXI COLECTIVO RURAL,FPCP93,2013-05-03,HYUNDAI,ACCENT RB GL 1.6,2013.0,4.0,...,Vehículos,NaN,NaN,NaN,TAXI COLECTIVO RURAL,hyundai,accent_rb_gl_1_6,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7827195,10480776,16.0,10036,AUTOMOVIL URBANO TAXI BÁSICO,BCSJ39,2008-02-13-00.00.00.000000,NISSAN,V16 EX SALOON 1.6,2008.0,4.0,...,Vehiculos,NaN,NaN,NaN,TAXI BÁSICO,nissan,v16_ex_saloon_1_6,NaN,NaN,NaN
7827196,10480777,16.0,39,AUTOMOVIL URBANO TAXI COLECTIVO,FDXL32,2014-05-26-00.00.00.000000,TOYOTA,NEW YARIS SEDAN GLI 1.5,2014.0,4.0,...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris_sedan_gli_1_5,NaN,NaN,NaN
7827197,10480778,16.0,94,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS66,2019-11-29-00.00.00.000000,PEUGEOT,301 ALLURE HDI,2020.0,NaN,...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,peugeot,301_allure_hdi,NaN,NaN,NaN
7827198,10480779,16.0,127,AUTOMOVIL URBANO TAXI COLECTIVO,KCZS65,2019-11-29-00.00.00.000000,TOYOTA,NEW YARIS,2020.0,NaN,...,Vehiculos,NaN,NaN,NaN,TAXI COLECTIVO URBANO,toyota,new_yaris,NaN,NaN,NaN


In [ ]:
vehiculos.combustible_final.isnull().sum()

5975316

In [ ]:
# graficar los años en los que hay valores nulos en combustible_final (histplot)
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
vehiculos[vehiculos['combustible_final'].isnull()]['ano_archivo'].hist(bins=20)

KeyError: 'ano_archivo'

<Figure size 1000x600 with 0 Axes>

In [ ]:
vehiculos.combustible_final.isnull().sum()

5975316

In [ ]:
asdasd

NameError: name 'asdasd' is not defined